# Global Imports

In [4]:
import os
from pprint import pprint

from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

True

We are using `load_dotenv(find_dotenv())` to find and load `.env` file. <br>
Why? 🤔 <br> 
If your environemnt file is not located in the root of your project we can still access it. <br>

# LLMs
LLMs [documentation](https://python.langchain.com/docs/modules/model_io/llms).

In [42]:
from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.messages import HumanMessage

In [73]:
llm = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"], organization=os.environ["OPENAI_ORGANIZATION"]
)
chat_model = ChatOpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
    organization=os.environ["OPENAI_ORGANIZATION"],
    model="gpt-3.5-turbo",
)

The LLM objects take string as input and output string. <br>
The ChatModel objects take a list of messages as input and output a message. <br>
For a deeper conceptual explanation of this difference please see [this documentation](https://python.langchain.com/docs/modules/model_io/concepts). <br>

In [74]:
text = "Are you an Alien?"
messages = [HumanMessage(content=text)]

response = llm.invoke(text)
pprint(f"LLM response: {response}")
pprint(f"LLM response type: {type(response)}")

pprint("-" * 100)

response = chat_model.invoke(messages)
pprint(f"Chat Model response: {response}")
pprint(f"Chat Model response type: {type(response)}")

'LLM response: \n\nNo, I am a digital AI created by humans.'
"LLM response type: <class 'str'>"
'----------------------------------------------------------------------------------------------------'
("Chat Model response: content='No, I am an artificial intelligence created by "
 "humans to assist with answering questions and providing information.' "
 "response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': "
 "12, 'total_tokens': 31}, 'model_name': 'gpt-3.5-turbo', "
 "'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': "
 'None}')
"Chat Model response type: <class 'langchain_core.messages.ai.AIMessage'>"


The problem with raw LLMs is that they don’t remember the history of the conversations

In [50]:
response = chat_model.invoke("What was my previous question?")
pprint(response)

AIMessage(content='Your previous question was "What is your favorite color?"', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 13, 'total_tokens': 24}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})


# Chains
Chains [documentation](https://python.langchain.com/docs/modules/chains).

## Add memory manually 
Following their current documentation and supported modules we can create a memory modeul and add history manually

In [98]:
from operator import itemgetter

from langchain.globals import set_debug, set_verbose
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

To see chain's process of toughts we neeed to activate **debugging** mode and **verbose**.
> 📎 **Note**: For some reason **verbose** is not working so we can use debug mode. Debug mode outputs a lot of text that is not important to us now so we will continu using basic output.

In [129]:
set_debug(value=False)
set_verbose(value=True)

In [122]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

In [123]:
memory = ConversationBufferMemory(return_messages=True)
memory.load_memory_variables({})

{'history': []}

In [124]:
chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | prompt
    | chat_model
)

In [125]:
inputs = {"input": "Are you an Alien?"}

response = chain.invoke(inputs)
response

AIMessage(content='No, I am not an alien. I am a computer program designed to assist and provide information to users. How can I help you today?', response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 22, 'total_tokens': 51}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

In [126]:
memory.save_context(inputs, {"output": response.content})

In [127]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Are you an Alien?'),
  AIMessage(content='No, I am not an alien. I am a computer program designed to assist and provide information to users. How can I help you today?')]}

In [128]:
chain.invoke({"input": "What was my previous question?"})

AIMessage(content='Your previous question was "Are you an Alien?" Is there anything else you would like to ask or discuss?', response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 65, 'total_tokens': 87}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

## Add memory automatically
We can use chain class that support automatic history tracking but they may be unsupported in the future.

Here we can set `verbose` in chain directly and it will work.

IMO this approach is better.

In [10]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI

In [11]:
chat_model = ChatOpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
    organization=os.environ["OPENAI_ORGANIZATION"],
    temperature=0,
)

conversation = ConversationChain(
    llm=chat_model, verbose=True, memory=ConversationBufferMemory()
)

In [12]:
conversation.predict(input="Are you an Alien?!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Are you an Alien?!
AI:

> Finished chain.


'No, I am not an alien. I am an artificial intelligence created by humans to assist with various tasks and provide information. I do not have a physical form like an alien would.'

As part of the prompt, we see now that there is a bit more then our simple prompt. <br>
We have a system prompt, the history of the conversation, and the human’s question. <br>
The system prompt allows us to give the LLM more context on what needs to be done. <br>
Let’s see if it remembers: <br>

In [13]:
conversation.predict(input="What was my previous question?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Are you an Alien?!
AI: No, I am not an alien. I am an artificial intelligence created by humans to assist with various tasks and provide information. I do not have a physical form like an alien would.
Human: What was my previous question?
AI:

> Finished chain.


'Your previous question was "Are you an Alien?!"'

# Prompt Templates
Prompt template [documetation](https://python.langchain.com/docs/modules/model_io/prompts/).

With LangChain, we can make the process of creating prompts easier by using prompt templates.

Let's make a simple prompt template:

In [15]:
from langchain.prompts import PromptTemplate

In [17]:
template = """
Return all the subcategories of the following category

{category}
"""

prompt = PromptTemplate(input_variables=["category"], template=template)

prompt

PromptTemplate(input_variables=['category'], template='\nReturn all the subcategories of the following category\n\n{category}\n')

"Category" serves as an input variable activated upon executing the chain.

Proceed to incorporate it into a chain. For this purpose, we employ the LLMChain, which stands as the most fundamental chain available for use.

In [18]:
from langchain.chains import LLMChain

In [21]:
chain = LLMChain(llm=chat_model, prompt=prompt, verbose=True)

chain.run("Machine Learning")



> Entering new LLMChain chain...
Prompt after formatting:

Return all the subcategories of the following category

Machine Learning


> Finished chain.


'- Supervised Learning\n- Unsupervised Learning\n- Reinforcement Learning\n- Deep Learning\n- Natural Language Processing\n- Computer Vision\n- Clustering\n- Classification\n- Regression\n- Dimensionality Reduction'

We can also split the prompts into **system** and **human** categories. 

This approach is useful when creating chatbots.

In [22]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

In [33]:
system_template = """
You are a helpful assistant who generate comma separated lists.
A user will only pass a category and you should generate subcategories of that category in a comma separated list.
ONLY return comma separated and nothing more!
"""

human_template = "{category}"

system_message = SystemMessagePromptTemplate.from_template(system_template)
human_message = HumanMessagePromptTemplate.from_template(human_template)

And we can combine the 2 prompts into one:

In [34]:
prompt = ChatPromptTemplate.from_messages(messages=[system_message, human_message])
prompt

ChatPromptTemplate(input_variables=['category'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='\nYou are a helpful assistant who generate comma separated lists.\nA user will only pass a category and you should generate subcategories of that category in a comma separated list.\nONLY return comma separated and nothing more!\n')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['category'], template='{category}'))])

In [35]:
chain = LLMChain(llm=chat_model, prompt=prompt, verbose=True)

response = chain.run("Machine Learning")
response



> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a helpful assistant who generate comma separated lists.
A user will only pass a category and you should generate subcategories of that category in a comma separated list.
ONLY return comma separated and nothing more!

Human: Machine Learning

> Finished chain.


'Supervised Learning, Unsupervised Learning, Reinforcement Learning, Deep Learning, Natural Language Processing, Computer Vision, Clustering, Classification, Regression, Dimensionality Reduction'

This approach allows us to better manage the output from the LLM, but we can enhance this further by using an output parser.

# Output  parser
Output parser [documentation](https://python.langchain.com/docs/modules/model_io/output_parsers/).


Let’s overwrite the base output parser and generate Python lists from the LLM’s response:

In [36]:
from langchain.schema import BaseOutputParser

In [37]:
class CommaSeparatedParser(BaseOutputParser):
    def parse(self, text):
        output = text.strip().split(",")
        output = [element.strip() for element in output]
        return output

In [38]:
chain = LLMChain(
    llm=chat_model, prompt=prompt, output_parser=CommaSeparatedParser(), verbose=True
)

In [39]:
response = chain.run("Machine Learning")

pprint(response)



> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a helpful assistant who generate comma separated lists.
A user will only pass a category and you should generate subcategories of that category in a comma separated list.
ONLY return comma separated and nothing more!

Human: Machine Learning

> Finished chain.
['Supervised Learning',
 'Unsupervised Learning',
 'Reinforcement Learning',
 'Deep Learning',
 'Natural Language Processing',
 'Computer Vision',
 'Clustering',
 'Classification',
 'Regression',
 'Dimensionality Reduction']


We can also pass to the chain multiple inputs at once with `apply` method:

In [41]:
input_list = [
    {"category": "food"},
    {"category": "drinks"},
    {"category": "colors"},
]
response = chain.apply(input_list)

pprint(response)



> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a helpful assistant who generate comma separated lists.
A user will only pass a category and you should generate subcategories of that category in a comma separated list.
ONLY return comma separated and nothing more!

Human: food
Prompt after formatting:
System: 
You are a helpful assistant who generate comma separated lists.
A user will only pass a category and you should generate subcategories of that category in a comma separated list.
ONLY return comma separated and nothing more!

Human: drinks
Prompt after formatting:
System: 
You are a helpful assistant who generate comma separated lists.
A user will only pass a category and you should generate subcategories of that category in a comma separated list.
ONLY return comma separated and nothing more!

Human: colors

> Finished chain.
[{'text': ['fruits', 'vegetables', 'grains', 'dairy', 'protein', 'sweets']},
 {'text': ['alcoholic',
           'non-alcoholic

Get only colors:

In [44]:
response[2]["text"]

['red',
 'orange',
 'yellow',
 'green',
 'blue',
 'indigo',
 'violet',
 'pink',
 'purple',
 'black',
 'white',
 'gray',
 'brown']

# Simple Sequence
Sequential chains [documentation](https://python.langchain.com/docs/modules/chains/foundational/sequential_chains).

We can also link chains together, forming a sequence where the output from one chain serves as the input for the next. 

To automate blog writing, we set up two chains: a title chain and a sections chain. Let's start with the title chain:

In [45]:
title_template = """
You are a writer. 
Given a subject, your job is to return the best title for a blog. 
Subject: {subject}
Title:
"""

title_chain = LLMChain.from_string(llm=chat_model, template=title_template)

title_chain.run("Machine Learning")

'"Unraveling the Wonders of Machine Learning: A Beginner\'s Guide"'

In [48]:
sections_template = """
You are a writer. 
Given a title, write sections for a blog.
Title: {title}
Synopsis:
"""

sections_chain = LLMChain.from_string(llm=chat_model, template=sections_template)

title = "Unraveling the Wonders of Machine Learning: A Beginner's Guide"

pprint(sections_chain.run(title))

('In this blog, we will delve into the fascinating world of machine learning, '
 'a rapidly growing field that is revolutionizing industries and changing the '
 'way we interact with technology. Whether you are a complete novice or have '
 "some basic knowledge of the subject, this beginner's guide will provide you "
 'with a comprehensive overview of machine learning and its applications.\n'
 '\n'
 'Section 1: What is Machine Learning?\n'
 'Machine learning is a subset of artificial intelligence that focuses on the '
 'development of algorithms and statistical models that enable computers to '
 'learn and make decisions without being explicitly programmed. In simple '
 'terms, it is the process of teaching a computer to recognize patterns in '
 'data and make predictions based on those patterns. This allows machines to '
 'improve their performance over time without human intervention.\n'
 '\n'
 'Section 2: Types of Machine Learning\n'
 'There are three main types of machine learning:

Let’s now combine those two chains by passing them as a list to the simple sequential chain:

In [49]:
from langchain.chains import SimpleSequentialChain

In [50]:
chain = SimpleSequentialChain(chains=[title_chain, sections_chain], verbose=True)

pprint(chain.run("Machine Learning"))



> Entering new SimpleSequentialChain chain...
"Unraveling the Wonders of Machine Learning: A Comprehensive Guide"
In this blog, we will delve into the fascinating world of machine learning, exploring its various applications, techniques, and potential impact on society. From self-driving cars to personalized recommendations on streaming platforms, machine learning is revolutionizing the way we interact with technology. Join us as we unravel the wonders of this cutting-edge field and discover how it is shaping the future of artificial intelligence.

Section 1: Introduction to Machine Learning
Machine learning is a subset of artificial intelligence that focuses on developing algorithms and models that can learn from and make predictions or decisions based on data. By analyzing patterns and trends in large datasets, machine learning algorithms can identify insights and make predictions without being explicitly programmed to do so. This section will provide an overview of the different t

This may be an interesting blog!

This concludes our introduction to LangChain basics. 

I hope you had some fun and learnd something new. 